* movie reviews NLTK movie_reviews
* subjectivitiydataset: nltk subjectivity

In [51]:
import torch
from torchtext.datasets import IMDB
from torchtext import data, datasets
import os
import pandas as pd

In [25]:
def load_doc(filename,encoding='utf8'):
    # open the file as read only
    file = open(filename, 'r',encoding=encoding)
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [31]:
def process_doc(directory):
    list_sentences = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        path = directory+'/'+filename
        list_sentences.append(load_doc(path))
    return list_sentences

In [43]:
sub_path_train_postive='dataset/train/pos/'
sub_path_train_negative='dataset/train/neg/'
sub_path_test_postive='dataset/test/pos/'
sub_path_test_negative='dataset/test/neg/'

In [45]:
training_postive=process_doc(sub_path_train_postive)
training_negative=process_doc(sub_path_train_negative)
testing_postive=process_doc(sub_path_test_postive)
testing_negative=process_doc(sub_path_test_negative)

In [89]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np

In [81]:
vectorizer = CountVectorizer()
classifier = SVC()

In [82]:
train_reviews=training_postive+training_negative
test_reviews=testing_postive+testing_negative

In [83]:
vectors = vectorizer.fit_transform(train_reviews+test_reviews)
train_vectors = vectors[:len(test_reviews)]
test_vectors = vectors[len(test_reviews):]

In [86]:
train_vectors.shape

(25000, 101895)

In [91]:
labels_training = np.array([0] * len(training_postive) + [1] * len(training_negative))
labels_testing = np.array([0] * len(testing_postive) + [1] * len(testing_negative))

In [92]:
classifier.fit(train_vectors, labels_training)
test_labels_predictions = classifier.predict(test_vectors)

print(classification_report(labels_testing, test_labels_predictions, digits=3))

              precision    recall  f1-score   support

           0      0.854     0.873     0.864     12500
           1      0.870     0.851     0.861     12500

    accuracy                          0.862     25000
   macro avg      0.862     0.862     0.862     25000
weighted avg      0.862     0.862     0.862     25000



In [55]:
dataset = pd.concat([pd.DataFrame({'review': training_postive, 'label':1}),
                     pd.DataFrame({'review': testing_postive, 'label':1}),
                     pd.DataFrame({'review': training_negative, 'label':0}),
                     pd.DataFrame({'review': testing_negative, 'label':0})],
                     axis=0, ignore_index=True)

In [56]:
dataset

,review,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1
...,...,...
49995,This is by far the worst version of William Sh...,0
49996,It's so depressing when film makers try to be ...,0
49997,"When I voted my ""1"" for this film I noticed th...",0
49998,This movie was so frustrating to watch. The sp...,0


In [54]:
dataset.tail()


,review,label
49995,This is by far the worst version of William Sh...,0
49996,It's so depressing when film makers try to be ...,0
49997,"When I voted my ""1"" for this film I noticed th...",0
49998,This movie was so frustrating to watch. The sp...,0
49999,I saw this film without knowing much about it ...,0


In [57]:
dataset.label.value_counts()


1    25000
0    25000
Name: label, dtype: int64

In [58]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [59]:
dataset.isna().sum()


review    0
label     0
dtype: int64

In [60]:
duplicate_indices = dataset.loc[dataset.duplicated(keep='first')].index
print('Number of duplicates in the dataset: {}'.format(dataset.loc[duplicate_indices, 'review'].count()))


Number of duplicates in the dataset: 418


In [61]:
dataset.loc[duplicate_indices, :].head()


,review,label
189,Though structured totally different from the b...,1
1673,Everyone knows about this ''Zero Day'' event. ...,1
2381,I Enjoyed Watching This Well Acted Movie Very ...,1
3018,"While there aren't any talking animals, big la...",1
4525,Wow! So much fun! Probably a bit much for norm...,1


In [62]:
dataset.drop_duplicates(keep='first', inplace=True)


In [63]:
print('Dataset shape after removing duplicates: {}'.format(dataset.shape))


Dataset shape after removing duplicates: (49582, 2)


In [65]:
dataset.to_csv('dataset_raw.csv', index=False)
